<a href="https://colab.research.google.com/github/Vaibhav-sa30/Channel-Effectiveness-in-HR-Data/blob/main/Channel_Effectiveness_in_HR_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime
from google.colab import files

# Upload the Excel file
uploaded = files.upload()

Saving Assignment 3 DataSet for Students.xlsx to Assignment 3 DataSet for Students (5).xlsx


# Run to get the rank of Applicants

In [ ]:
# Load the Excel file into a DataFrame
file_name = list(uploaded.keys())[0]
df = pd.read_excel(file_name, skiprows=0, nrows=11, usecols="A:H")

# Convert the 'Appraisal history' column to a list of lists of floats
df['Appraisal history'] = df['Appraisal history'].apply(lambda x: [float(val) for val in x.split(', ')])
df['Count Appraisal > 0.7'] = df['Appraisal history'].apply(lambda x: sum(1 for val in x if val > 0.7))

# Convert the 'When the candidate will be available' column to datetime
current_date = datetime.strptime('2023-08-04', '%Y-%m-%d')
df['When the candidate will be available'] = pd.to_datetime(df['When the candidate will be available'], format='%m/%d/%Y')
df['Days Until Availability'] = (df['When the candidate will be available'] - current_date).dt.days

# Define the maximum values for normalization
max_experience = df['Year of experience'].max()
max_count_appraisal = df['Count Appraisal > 0.7'].max()
max_skills_certifications = df['Skills/Certifications'].apply(lambda x: len(x.split(', '))).max()
max_key_projects = df['Key projects'].apply(lambda x: len(x.split(', '))).max()
max_duration_current_role = df['Duration in the current role'].max()
max_bench_duration = df['Bench duration'].max()
max_days_until_availability = df['Days Until Availability'].max()

# Normalize each criterion
df['Normalized Year of experience'] = df['Year of experience'] / max_experience
df['Normalized Count Appraisal > 0.7'] = df['Count Appraisal > 0.7'] / max_count_appraisal
df['Normalized Skills/Certifications'] = df['Skills/Certifications'].apply(lambda x: len(x.split(', ')) / max_skills_certifications)
df['Normalized Key projects'] = df['Key projects'].apply(lambda x: len(x.split(', ')) / max_key_projects)
df['Normalized Duration in the current role'] = df['Duration in the current role'] / max_duration_current_role
df['Normalized Bench duration'] = df['Bench duration'] / max_bench_duration
df['Normalized Days Until Availability'] = (max_days_until_availability - df['Days Until Availability']) / max_days_until_availability

# Calculate the weighted average score
df['Average Score'] = df[
    ['Normalized Year of experience', 'Normalized Count Appraisal > 0.7', 'Normalized Skills/Certifications',
     'Normalized Key projects', 'Normalized Duration in the current role', 'Normalized Bench duration',
     'Normalized Days Until Availability']
].mean(axis=1)

# Rank the candidates based on their scores
df = df.sort_values(by='Average Score', ascending=False).reset_index(drop=True)
df['Rank'] = df.index + 1

# Display the results
print(df[['Employee name', 'Rank']])


  Employee name  Rank
0         Gauri     1
1         Divya     2
2        Bhavya     3
3         Arjun     4
4          Esha     5
5        Farhan     6
6     Chaitanya     7
7        Ishita     8
8        Harish     9
9           Jai    10


# Run to get Rank of Channels

In [ ]:
df = pd.read_excel(file_name, sheet_name="Application Details")

# Convert feedback column to numeric
df['Applicants feedback on Ease of application'] = pd.to_numeric(df['Applicants feedback on Ease of application'], errors='coerce')

# Calculate the percent of applications received by channel
art = df.pivot_table(index='Channel', values='Application number', aggfunc='count', fill_value=0)
art['Percent of Application received'] = art['Application number'] / art['Application number'].sum()

# Average Weekly Flow Table
awft = df.pivot_table(index='Channel', columns='Week', values='Application number', aggfunc='count', fill_value=0)
awft['Average Weekly Flow'] = awft.mean(axis=1)

# Percent Shortlisted
st = df.pivot_table(index='Channel', columns='Shortlisted ', values='Application number', aggfunc='count', fill_value=0)
st['Percent Shortlisted'] = st['Yes'] / (st['Yes'] + st['No'])

# Applicants feedback on Ease of Application
feedback_pivot = df.pivot_table(index='Channel', values='Applicants feedback on Ease of application', aggfunc='mean', fill_value=0)

# Find the channel with the highest average rating
highest_rating_channel = feedback_pivot['Applicants feedback on Ease of application'].idxmax()

# Combine all tables into a single DataFrame
table = pd.concat([art, awft, st, feedback_pivot], axis=1)
table['Normalised Average Weekly Flow'] = table['Average Weekly Flow']/table['Average Weekly Flow'].max()
table['Normalised Applicants Feedback'] = table['Applicants feedback on Ease of application']/table['Applicants feedback on Ease of application'].max()
# Display the channel with the highest average rating
print("Channel with the highest average rating:", highest_rating_channel)

# Calculate the average score for ranking
table['Average Score'] = table[
    ['Percent of Application received', 'Normalised Average Weekly Flow','Percent Shortlisted','Normalised Applicants Feedback']
].mean(axis=1)

# Display the results
# Rank the table by average score
table_ranked = table.sort_values(by='Average Score', ascending=False)
table_ranked['Rank'] = range(1, len(table_ranked) + 1)


# Print channel and rank
table_ranked[['Rank']].reset_index()

Channel with the highest average rating: Employee referral


,Channel,Rank
0,Third party,1
1,Employee referral,2
2,Direct website,3
3,LinkedIn,4


In [ ]:
table[
    ['Percent of Application received', 'Normalised Average Weekly Flow','Percent Shortlisted','Normalised Applicants Feedback','Average Score']
]

,Percent of Application received,Normalised Average Weekly Flow,Percent Shortlisted,Normalised Applicants Feedback,Average Score
Channel,,,,,
Direct website,0.203655,0.534247,0.628205,0.900283,0.566597
Employee referral,0.201044,0.527397,0.558442,1.000000,0.571721
LinkedIn,0.214099,0.561644,0.536585,0.915632,0.556990
Third party,0.381201,1.000000,0.684932,0.899916,0.741512
